# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file into a DataFrame
recipes = pd.read_csv("recipes.tsv", sep="\t")
recipes

,recipe_slug,recipe_title,recipe_introduction
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...
...,...,...,...
95,bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...
96,chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...
97,classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...
98,vegetarian-mushroom-lasagna,Vegetarian Mushroom Lasagna,Vegetarian mushroom lasagna is a delicious and...


In [2]:
recipe_tags = pd.read_csv("recipe-tags.tsv", sep="\t")
recipe_tags

,recipe_slug,recipe_tag
0,spam-musubi,hawaiian
1,spam-musubi,nori
2,spam-musubi,onthego
3,spam-musubi,rice
4,spam-musubi,snack
...,...,...
747,bacon-souffle,breakfast
748,bacon-souffle,brunch
749,bacon-souffle,cheese
750,bacon-souffle,eggs


In [3]:
# Group tags so each recipe_slug has one row with a list of tags
tags_per_recipe = (
    recipe_tags
    .groupby("recipe_slug")["recipe_tag"]
    .apply(list)
    .reset_index(name="recipe_tags")
)

# Merge onto the recipes table (left join keeps all recipes)
recipes_merged = recipes.merge(tags_per_recipe, on="recipe_slug", how="left")

# If you want recipes with no tags to have an empty list instead of NaN
recipes_merged["recipe_tags"] = recipes_merged["recipe_tags"].apply(
    lambda value: value if isinstance(value, list) else []
)

recipes_merged


,recipe_slug,recipe_title,recipe_introduction,recipe_tags
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...,"[appetizer, middleeastern, snack, streetfood, ..."
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...,"[cheese, fried, grilled, hamburger, hawaii, ke..."
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...,"[bacon, breakfast, dinner, eggs, filling, frie..."
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...,"[appetizer, chicken, crispy, fingerfood, fried..."
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...,"[apple, baked, comfortfood, crisp, crumble, de..."
...,...,...,...,...
95,bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...,"[bacon, cheese, comfortfood, creamy, macandche..."
96,chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...,"[alfredo, chicken, comfortfood, italiancuisine..."
97,classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...,"[cheesy, comfortfood, groundbeef, italian, pas..."
98,vegetarian-mushroom-lasagna,Vegetarian Mushroom Lasagna,Vegetarian mushroom lasagna is a delicious and...,"[comfortfood, italian, lasagna, mushroom, past..."


In [4]:
# List number of times tags appear across all recipes
from collections import Counter

# Count the occurrences of each tag
tag_counts = Counter(tag for tags in recipes_merged["recipe_tags"] for tag in tags)

# Convert to DataFrame for easier visualization
tag_counts_df = pd.DataFrame(tag_counts.items(), columns=["tag", "count"])

# Display the most common tags
tag_counts_df.sort_values("count", ascending=False)

,tag,count
37,dessert,28
18,breakfast,20
6,cheese,18
5,vegetarian,17
34,comfortfood,16
...,...,...
159,customizable,1
160,handheld,1
161,cupcakes,1
163,indulgent,1


In [5]:
# Assign ratings 1-5 based on tags

# If the recipe has "cheese" in "recipe_tags", assign a rating of 5
# If the recipe has "dessert" in "recipe_tags", assign a rating of 1
# Otherwise, randomly assign a rating of 2, 3, or 4
import random
def assign_rating(tags):
    if "cheese" in tags:
        return 5
    elif "dessert" in tags:
        return 1
    else:
        return random.choice([2, 3, 4])
recipes_merged["rating"] = recipes_merged["recipe_tags"].apply(assign_rating)
recipes_merged


,recipe_slug,recipe_title,recipe_introduction,recipe_tags,rating
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...,"[appetizer, middleeastern, snack, streetfood, ...",4
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...,"[cheese, fried, grilled, hamburger, hawaii, ke...",5
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...,"[bacon, breakfast, dinner, eggs, filling, frie...",3
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...,"[appetizer, chicken, crispy, fingerfood, fried...",2
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...,"[apple, baked, comfortfood, crisp, crumble, de...",1
...,...,...,...,...,...
95,bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...,"[bacon, cheese, comfortfood, creamy, macandche...",5
96,chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...,"[alfredo, chicken, comfortfood, italiancuisine...",2
97,classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...,"[cheesy, comfortfood, groundbeef, italian, pas...",2
98,vegetarian-mushroom-lasagna,Vegetarian Mushroom Lasagna,Vegetarian mushroom lasagna is a delicious and...,"[comfortfood, italian, lasagna, mushroom, past...",2


In [6]:
ratings = pd.DataFrame({
    "recipe_slug": recipes_merged["recipe_slug"],
    "rating": (recipes_merged["rating"] - 1) * 0.25 #convert between 1-5 to 0-1
})
ratings.to_csv("ratings.tsv", sep="\t", index=False)

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [7]:
# YOUR CHANGES HERE
features = pd.DataFrame({
    "recipe_slug": recipes_merged["recipe_slug"],
    "bias": 1
})
# Column for each tag, 1 if the recipe has that tag, 0 otherwise
for tag in tag_counts.keys():
    features[tag] = recipes_merged["recipe_tags"].apply(lambda tags: 1 if tag in tags else 0)

features.to_csv("features.tsv", sep="\t", index=False)

/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/799432347.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[tag] = recipes_merged["recipe_tags"].apply(lambda tags: 1 if tag in tags else 0)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [8]:
# YOUR CHANGES HERE
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load features and ratings
features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

# Merge features and ratings on recipe_slug
data = features.merge(ratings, on="recipe_slug")

# Separate features and target variable
X = data.drop(columns=["recipe_slug", "rating"])
y = data["rating"]

# Train a Ridge regression model
ridge_model = Ridge(alpha=1.0, fit_intercept=False) # fit_intercept=False because we have a bias feature
ridge_model.fit(X, y)

/Users/chandlercampbell/.venvs/ds-gpu/lib/python3.12/site-packages/sklearn/utils/extmath.py:227: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/chandlercampbell/.venvs/ds-gpu/lib/python3.12/site-packages/sklearn/utils/extmath.py:227: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/chandlercampbell/.venvs/ds-gpu/lib/python3.12/site-packages/sklearn/utils/extmath.py:227: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


,"alpha alpha: {float, ndarray of shape (n_targets,)}, default=1.0Constant that multiplies the L2 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Ridge` object is not advised.Instead, you should use the :class:`LinearRegression` object.If an array is passed, penalties are assumed to be specific to thetargets. Hence they must correspond in number.",1.0
,"fit_intercept fit_intercept: bool, default=TrueWhether to fit the intercept for this model. If setto false, no intercept will be used in calculations(i.e. ``X`` and ``y`` are expected to be centered).",False
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=NoneMaximum number of iterations for conjugate gradient solver.For 'sparse_cg' and 'lsqr' solvers, the default value is determinedby scipy.sparse.linalg. For 'sag' solver, the default value is 1000.For 'lbfgs' solver, the default value is 15000.",None
,"tol tol: float, default=1e-4The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for each solver:- 'svd': `tol` has no impact.- 'cholesky': `tol` has no impact.- 'sparse_cg': norm of residuals smaller than `tol`.- 'lsqr': `tol` is set as atol and btol of scipy.sparse.linalg.lsqr, which control the norm of the residual vector in terms of the norms of matrix and coefficients.- 'sag' and 'saga': relative change of coef smaller than `tol`.- 'lbfgs': maximum of the absolute (projected) gradient=max|residuals| smaller than `tol`... versionchanged:: 1.2 Default value changed from 1e-3 to 1e-4 for consistency with other linear models.",0.0001
,"solver solver: {'auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'}, default='auto'Solver to use in the computational routines:- 'auto' chooses the solver automatically based on the type of data.- 'svd' uses a Singular Value Decomposition of X to compute the Ridge coefficients. It is the most stable solver, in particular more stable for singular matrices than 'cholesky' at the cost of being slower.- 'cholesky' uses the standard :func:`scipy.linalg.solve` function to obtain a closed-form solution.- 'sparse_cg' uses the conjugate gradient solver as found in :func:`scipy.sparse.linalg.cg`. As an iterative algorithm, this solver is more appropriate than 'cholesky' for large-scale data (possibility to set `tol` and `max_iter`).- 'lsqr' uses the dedicated regularized least-squares routine :func:`scipy.sparse.linalg.lsqr`. It is the fastest and uses an iterative procedure.- 'sag' uses a Stochastic Average Gradient descent, and 'saga' uses its improved, unbiased version named SAGA. Both methods also use an iterative procedure, and are often faster than other solvers when both n_samples and n_features are large. Note that 'sag' and 'saga' fast convergence is only guaranteed on features with approximately the same scale. You can preprocess the data with a scaler from :mod:`sklearn.preprocessing`.- 'lbfgs' uses L-BFGS-B algorithm implemented in :func:`scipy.optimize.minimize`. It can be used only when `positive` is True.All solvers except 'svd' support both dense and sparse data. However, only'lsqr', 'sag', 'sparse_cg', and 'lbfgs' support sparse input when`fit_intercept` is True... versionadded:: 0.17 Stochastic Average Gradient descent solver... versionadded:: 0.19 SAGA solver.",'auto'
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive.Only 'lbfgs' solver is supported in this case.",False
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag' or 'saga' to shuffle the data.See :term:`Glossary ` for details... versionadded:: 0.17 `random_state` to support Stochastic Average Gradient.",None


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [9]:
# YOUR CHANGES HERE
model = pd.DataFrame({
    "recipe_tag": X.columns,
    "coefficient": ridge_model.coef_
})
model.to_csv("model.tsv", sep="\t", index=False)

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [10]:
# Make predictions
score_estimate = ridge_model.predict(X)
estimates = pd.DataFrame({
    "recipe_slug": data["recipe_slug"],
    "score_estimate": score_estimate
})
estimates.to_csv("estimates.tsv", sep="\t", index=False)


/Users/chandlercampbell/.venvs/ds-gpu/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/chandlercampbell/.venvs/ds-gpu/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/chandlercampbell/.venvs/ds-gpu/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [11]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

ridge_lambda = 1
alpha = 2

features_df = pd.read_csv("features.tsv", sep="\t")
ratings_df = pd.read_csv("ratings.tsv", sep="\t")

merged_df = features_df.merge(ratings_df, on="recipe_slug", how="inner")

feature_columns = [col for col in merged_df.columns if col not in ["recipe_slug", "rating"]]

X_matrix = merged_df[feature_columns].to_numpy(dtype=float)
reward_vector = merged_df["rating"].to_numpy(dtype=float)

num_features = X_matrix.shape[1]

A_matrix = ridge_lambda * np.eye(num_features, dtype=float)
b_vector = np.zeros(num_features, dtype=float)

for i in range(X_matrix.shape[0]):
    x_vector = X_matrix[i]
    r_value = reward_vector[i]
    A_matrix += np.outer(x_vector, x_vector)
    b_vector += r_value * x_vector

theta_vector = np.linalg.solve(A_matrix, b_vector)
A_inverse = np.linalg.inv(A_matrix)

predicted_mean = X_matrix @ theta_vector

variance = np.sum((X_matrix @ A_inverse) * X_matrix, axis=1)
variance = np.clip(variance, 0.0, None)

score_bound = predicted_mean + alpha * np.sqrt(variance)

bounds_df = pd.DataFrame({
    "recipe_slug": merged_df["recipe_slug"].values,
    "score_bound": score_bound
})

bounds_df.to_csv("bounds.tsv", sep="\t", index=False)

bounds_df.head(10)


/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/2341498743.py:33: RuntimeWarning: divide by zero encountered in matmul
  predicted_mean = X_matrix @ theta_vector
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/2341498743.py:33: RuntimeWarning: overflow encountered in matmul
  predicted_mean = X_matrix @ theta_vector
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/2341498743.py:33: RuntimeWarning: invalid value encountered in matmul
  predicted_mean = X_matrix @ theta_vector
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/2341498743.py:35: RuntimeWarning: divide by zero encountered in matmul
  variance = np.sum((X_matrix @ A_inverse) * X_matrix, axis=1)
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/2341498743.py:35: RuntimeWarning: overflow encountered in matmul
  variance = np.sum((X_matrix @ A_inverse) * X_matrix, axis=1)
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/2341498743.py:35: R

,recipe_slug,score_bound
0,falafel,2.469298
1,spamburger,2.872718
2,bacon-fried-rice,2.365262
3,chicken-fingers,2.077283
4,apple-crisp,1.797353
5,cranberry-apple-crisp,1.822293
6,bacon-chocolate-chip-cookies,1.706072
7,sujebi,2.102395
8,pasta-primavera,2.006189
9,ramen,2.173074


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

Hint: do not remove recipes after each recommendation.
Repeating recommendations is expected.

In [12]:


ridge_lambda = 1
alpha = 2
num_recommendations = 100

# Load the same data as Part 5
features_df = pd.read_csv("features.tsv", sep="\t")
ratings_df = pd.read_csv("ratings.tsv", sep="\t")

# Map recipe_slug -> rating (reward)
rating_map = dict(zip(ratings_df["recipe_slug"], ratings_df["rating"]))

feature_columns = [col for col in features_df.columns if col != "recipe_slug"]

X_matrix = features_df[feature_columns].to_numpy(dtype=float)
recipe_slugs = features_df["recipe_slug"].to_numpy()

num_features = X_matrix.shape[1]

# Start with no data: A = lambda * I, b = 0
A_inverse = (1.0 / ridge_lambda) * np.eye(num_features, dtype=float)
b_vector = np.zeros(num_features, dtype=float)

recommendation_rows = []

for t in range(num_recommendations):
    # theta = A^{-1} b
    theta_vector = A_inverse @ b_vector

    # LinUCB bounds for all recipes
    predicted_mean = X_matrix @ theta_vector
    variance = np.sum((X_matrix @ A_inverse) * X_matrix, axis=1)
    variance = np.clip(variance, 0.0, None)
    score_bounds = predicted_mean + alpha * np.sqrt(variance)

    # Pick best (ties broken arbitrarily by np.argmax)
    best_index = int(np.argmax(score_bounds))
    best_slug = recipe_slugs[best_index]
    best_bound = float(score_bounds[best_index])

    if best_slug not in rating_map:
        raise ValueError(f"No reward found for recipe_slug = {best_slug}")

    reward = float(rating_map[best_slug])

    # Record this recommendation
    recommendation_rows.append({
        "recipe_slug": best_slug,
        "score_bound": best_bound,
        "reward": reward
    })

    # Update LinUCB stats with this (x, reward)
    x_vector = X_matrix[best_index]

    # Sherman-Morrison update for A_inverse:
    # A_new = A + x x^T
    # A_inv_new = A_inv - (A_inv x x^T A_inv) / (1 + x^T A_inv x)
    A_inv_x = A_inverse @ x_vector
    denominator = 1.0 + float(x_vector @ A_inv_x)
    A_inverse = A_inverse - np.outer(A_inv_x, A_inv_x) / denominator

    # b_new = b + reward * x
    b_vector = b_vector + reward * x_vector

# Save recommendations.tsv
recommendations_df = pd.DataFrame(recommendation_rows)
recommendations_df.to_csv("recommendations.tsv", sep="\t", index=False)

/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/1848858214.py:27: RuntimeWarning: divide by zero encountered in matmul
  theta_vector = A_inverse @ b_vector
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/1848858214.py:27: RuntimeWarning: overflow encountered in matmul
  theta_vector = A_inverse @ b_vector
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/1848858214.py:27: RuntimeWarning: invalid value encountered in matmul
  theta_vector = A_inverse @ b_vector
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/1848858214.py:30: RuntimeWarning: divide by zero encountered in matmul
  predicted_mean = X_matrix @ theta_vector
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/1848858214.py:30: RuntimeWarning: overflow encountered in matmul
  predicted_mean = X_matrix @ theta_vector
/var/folders/rv/ykggspv13l50b7nthk6kwm6r0000gn/T/ipykernel_2491/1848858214.py:30: RuntimeWarning: invalid value encountered in matmul
  pr

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.